**Import necessary libraries**

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import logging
import os
import re
from matplotlib.backends.backend_pdf import PdfPages

**Configuration**

In [44]:
INPUT_FILENAME = 'online_advertising_performance_data.csv'
OUTPUT_FILENAME = 'top_actions.csv'
REPORT_FILENAME = 'Advertising_Performance_Report.pdf'
ANALYSIS_YEAR = 2020
SAVE_PLOTS = True  # Set to True to save all generated plots to the '/plots' directory

**Setup Logging**

In [45]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

**Setup Plotting Style**

In [46]:
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100
logging.info("Setup complete. Configuration loaded.")

**Helper Functions**

In [47]:
def setup_plot(title, xlabel=None, ylabel=None, figsize=(10, 6)):
    """
    Creates a matplotlib figure and axes with a consistent title and labels.

    Returns:
        matplotlib.axes.Axes: The plot axes for further manipulation.
    """
    plt.figure(figsize=figsize, dpi=100)
    ax = plt.gca()
    ax.set_title(title, fontsize=14, fontweight='bold')
    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)
    return ax

def finalize_plot(title, save_plots=False, tight_layout=True, pdf_pages=None):
    """
    Applies final layout adjustments, saves the plot if requested, and displays it.

    Args:
        title (str): The title of the plot, used for the filename.
        save_plots (bool): If True, saves the figure to the 'plots' directory.
        tight_layout (bool): If True, applies plt.tight_layout().
        pdf_pages (PdfPages object, optional): If provided, saves the figure to the PDF.
    """
    if tight_layout:
        plt.tight_layout()

    if pdf_pages:
        pdf_pages.savefig(plt.gcf())
        plt.close()
    else:
        if save_plots:
            output_dir = 'plots'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            safe_filename = re.sub(r'[^\w\s-]', '', title).strip().replace(' ', '_')
            filepath = os.path.join(output_dir, f"{safe_filename}.png")
            try:
                plt.savefig(filepath, bbox_inches='tight')
                logging.info(f"Plot saved: {filepath}")
            except Exception as e:
                logging.error(f"Failed to save plot {filepath}: {e}")
        plt.show()
        plt.close()

def add_text_to_pdf(text_content, pdf_pages_obj):
    """Renders a string of text onto a new page in the PDF report."""
    fig = plt.figure(figsize=(8.5, 11))
    fig.text(0.05, 0.95, text_content, ha='left', va='top', wrap=True, fontsize=8, fontfamily='monospace')
    pdf_pages_obj.savefig(fig)
    plt.close(fig)

def compute_kpis(data):
    """
    Computes CTR, CPC, RPC, CVR, AOV, and ROAS from base metrics.

    Note: A copy of the DataFrame is made to prevent unintended side effects
    on the original DataFrame, which is a safer practice.

    Args:
        data (pd.DataFrame): DataFrame with base metrics.

    Returns:
        pd.DataFrame: DataFrame with added KPI columns.
    """
    df_kpi = data.copy()
    # Use np.nan for division by zero to correctly calculate means later
    df_kpi['ctr'] = df_kpi['clicks'] / df_kpi['displays'].replace(0, np.nan)
    df_kpi['cpc'] = df_kpi['cost'] / df_kpi['clicks'].replace(0, np.nan)
    df_kpi['rpc'] = df_kpi['revenue'] / df_kpi['clicks'].replace(0, np.nan)
    df_kpi['cvr'] = df_kpi['conversions'] / df_kpi['clicks'].replace(0, np.nan)
    df_kpi['aov'] = df_kpi['sales'] / df_kpi['conversions'].replace(0, np.nan)
    df_kpi['roas'] = df_kpi['revenue'] / df_kpi['cost'].replace(0, np.nan)
    return df_kpi.fillna(0)

**Data Loading and Cleaning**

In [48]:
start_time = time.time()
logging.info(f"Loading data from {INPUT_FILENAME}...")

try:
    df = pd.read_csv(INPUT_FILENAME)
    logging.info("Data loaded successfully.")
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('#n/a', 'unknown', regex=False)
    # Remove empty 'unnamed' columns that can result from CSV export issues
    df = df.loc[:, ~df.columns.str.contains('^unnamed')]
    logging.info("Removed unnamed columns.")
    # Drop rows where essential date components are missing
    df.dropna(subset=['month', 'day'], inplace=True)
    # Parse dates using the configured year
    df['date'] = pd.to_datetime(f'{ANALYSIS_YEAR}-' + df['month'] + '-' + df['day'].astype(str), errors='coerce')
    df.dropna(subset=['date'], inplace=True) # Drop rows where date conversion failed
    # Handle missing and inconsistent placements
    df['placement'] = df['placement'].fillna('Unknown')
    df.loc[df['placement'] == 'unknown', 'placement'] = 'Unknown'
    # Convert numeric types, coercing errors to 0
    numeric_cols = ['displays', 'cost', 'clicks', 'revenue', 'post_click_conversions', 'post_click_sales_amount']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    # Handle any remaining nulls in object columns by filling with 'Unknown'
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna('Unknown')
    # Rename columns for clarity and convenience
    df.rename(columns={
        'post_click_conversions': 'conversions',
        'post_click_sales_amount': 'sales'
    }, inplace=True)

    # --- Data Validation ---
    logging.info("Validating data integrity...")
    assert df['cost'].min() >= 0, "Validation Failed: Cost contains negative values"
    assert df['displays'].min() >= 0, "Validation Failed: Displays contains negative values"
    assert df['clicks'].min() >= 0, "Validation Failed: Clicks contains negative values"
    assert df['revenue'].min() >= 0, "Validation Failed: Revenue contains negative values"
    assert df['conversions'].min() >= 0, "Validation Failed: Conversions contains negative values"
    assert df['sales'].min() >= 0, "Validation Failed: Sales contains negative values"
    assert df['date'].min() >= pd.to_datetime(f'{ANALYSIS_YEAR}-04-01'), "Validation Failed: Date range starts before April"
    assert df['date'].max() <= pd.to_datetime(f'{ANALYSIS_YEAR}-06-30'), "Validation Failed: Date range extends beyond June"

    # Final check for any remaining null values
    assert df.isnull().sum().sum() == 0, "Validation Failed: Null values still remain after cleaning"

    logging.info("Data cleaning and validation successful.")
    df.info()

except FileNotFoundError:
    logging.error(f"Error: The file '{INPUT_FILENAME}' was not found.")
    df = pd.DataFrame() # Create an empty df to prevent further errors

loading_time = time.time() - start_time
logging.info(f"Data Loading & Cleaning took {loading_time:.2f} seconds.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15408 entries, 0 to 15407
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   month            15408 non-null  object        
 1   day              15408 non-null  int64         
 2   campaign_number  15408 non-null  object        
 3   user_engagement  15408 non-null  object        
 4   banner           15408 non-null  object        
 5   placement        15408 non-null  object        
 6   displays         15408 non-null  int64         
 7   cost             15408 non-null  float64       
 8   clicks           15408 non-null  int64         
 9   revenue          15408 non-null  float64       
 10  conversions      15408 non-null  int64         
 11  sales            15408 non-null  float64       
 12  date             15408 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(4), object(5)
memory usage: 1.5+ MB


**KPI Engineering**

In [49]:
if not df.empty:
    start_time = time.time()

    df_kpi = compute_kpis(df)
    logging.info("KPIs (CTR, CPC, CVR, ROAS, etc.) computed and added to the DataFrame.")

    kpi_time = time.time() - start_time
    logging.info(f"KPI Engineering took {kpi_time:.2f} seconds.")

    df_kpi.head()

**Answering Business Questions**

In [40]:
if not df.empty:
    analysis_start_time = time.time()
    # Initialize the PDF report object
    pdf_report = PdfPages(REPORT_FILENAME)
    # Add a title page to the report
    title_page_text = (
        f"Online Advertising Performance Report\n\n"
        f"Date Range: April 2020 - June 2020\n"
        f"Generated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}"
    )
    add_text_to_pdf(title_page_text, pdf_report)

    #Q1: Trend in engagement over time?
    daily_engagement_revenue = df_kpi.groupby(['date', 'user_engagement'])['revenue'].sum().unstack().fillna(0)
    ax = setup_plot('Daily Revenue by User Engagement', ylabel='Total Revenue')
    daily_engagement_revenue.plot(ax=ax)
    finalize_plot('Daily Revenue by User Engagement', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q2: Banner size vs clicks?
    banner_clicks = df_kpi.groupby('banner')['clicks'].sum().sort_values(ascending=False)
    ax = setup_plot('Total Clicks by Banner Size', ylabel='Total Clicks')
    banner_clicks.plot(kind='bar', ax=ax)
    plt.xticks(rotation=45, ha='right')
    finalize_plot('Total Clicks by Banner Size', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q3: Top placements by displays and clicks?
    placement_summary = df_kpi.groupby('placement').agg(
        total_displays=('displays', 'sum'),
        total_clicks=('clicks', 'sum')
    ).sort_values('total_displays', ascending=False).head(5)
    q3_text = "Top 5 Placements by Volume \n" + placement_summary.to_markdown(floatfmt=".2f")
    logging.info(q3_text)
    add_text_to_pdf(q3_text, pdf_report)

    #Q4: Cost vs revenue correlation?
    correlation = df_kpi['cost'].corr(df_kpi['revenue'])
    title = f'Cost vs. Revenue (Correlation: {correlation:.2f})'
    ax = setup_plot(title, xlabel='Cost', ylabel='Revenue')
    sns.scatterplot(data=df_kpi, x='cost', y='revenue', alpha=0.5, ax=ax)
    finalize_plot('Cost vs Revenue', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q5: Average revenue per click?
    total_revenue = df_kpi['revenue'].sum()
    total_clicks = df_kpi['clicks'].sum()
    avg_rpc = total_revenue / total_clicks if total_clicks > 0 else 0
    q5_text = f"Overall Average Revenue Per Click (RPC): ${avg_rpc:.4f} ---"
    logging.info(q5_text)
    add_text_to_pdf(q5_text, pdf_report)

    #Q6: Highest post-click conversion rates by campaign?
    campaign_cvr = df_kpi.groupby('campaign_number')['cvr'].mean().sort_values(ascending=False)
    ax = setup_plot('Average Conversion Rate (CVR) by Campaign', ylabel='Average CVR')
    campaign_cvr.plot(kind='bar', ax=ax)
    plt.xticks(rotation=0)
    finalize_plot('Average Conversion Rate (CVR) by Campaign', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q7: Post-click sales trend?
    daily_sales = df_kpi.groupby('date')['sales'].sum()
    ax = setup_plot('Daily Post-Click Sales Amount', ylabel='Total Sales ($)')
    daily_sales.plot(ax=ax)
    finalize_plot('Daily Post-Click Sales Amount', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q8: Engagement by banner size?
    engagement_banner_cvr = df_kpi.groupby(['user_engagement', 'banner'])['cvr'].mean().unstack()
    ax = setup_plot('Heatmap of CVR by Engagement and Banner Size')
    sns.heatmap(engagement_banner_cvr, annot=True, fmt=".2%", cmap="YlGnBu", ax=ax)
    finalize_plot('Heatmap of CVR by Engagement and Banner Size', save_plots=SAVE_PLOTS, tight_layout=False, pdf_pages=pdf_report)

    # ### Q9: Placement with highest CVR?
    placement_cvr = df_kpi.groupby('placement')['cvr'].mean().sort_values(ascending=False).head(5)
    q9_text = "Top 5 Placements by Average CVR\n" + placement_cvr.to_markdown(floatfmt=".2%")
    logging.info(q9_text)
    add_text_to_pdf(q9_text, pdf_report)

    #Q10 & Q19: Seasonal and Day-of-week patterns?
    df_kpi['day_of_week'] = df_kpi['date'].dt.day_name()
    dow_patterns = df_kpi.groupby('day_of_week')[['displays', 'clicks', 'cvr']].mean().reindex(
        ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    )
    dow_patterns.plot(kind='bar', subplots=True, layout=(3,1), legend=False,
                      title='Average Daily Performance by Day of Week', figsize=(10, 8))
    finalize_plot('Average Daily Performance by Day of Week', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q11: Engagement vs revenue?
    engagement_revenue = df_kpi.groupby('user_engagement')['revenue'].sum().sort_values()
    ax = setup_plot('Share of Revenue by User Engagement')
    engagement_revenue.plot(kind='pie', autopct='%1.1f%%', ax=ax, startangle=90)
    ax.set_ylabel('') # Clean up pie chart
    finalize_plot('Share of Revenue by User Engagement', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q12: Outliers?
    cpc_std = df_kpi['cpc'].std()
    denominator = cpc_std if cpc_std > 0 else 1
    df_kpi['cpc_zscore'] = np.abs((df_kpi['cpc'] - df_kpi['cpc'].mean()) / denominator)
    outliers = df_kpi[df_kpi['cpc_zscore'] > 3]
    q12_text = f"Found {len(outliers)} records with CPC z-score > 3 (potential outliers)."
    logging.info(q12_text)
    add_text_to_pdf(q12_text, pdf_report)

    #Q13: Effectiveness by size and placement?
    roas_pivot = df_kpi.groupby(['banner', 'placement'])['roas'].mean().unstack().fillna(0)
    ax = setup_plot('Average ROAS by Banner and Placement', figsize=(8, 6))
    sns.heatmap(roas_pivot, cmap="Greens", annot=True, fmt=".1f", linewidths=.5, ax=ax)
    finalize_plot('Average ROAS by Banner and Placement', save_plots=SAVE_PLOTS, tight_layout=False, pdf_pages=pdf_report)

    #Q14: Consistent ROI by banner?
    median_roas_banner = df_kpi.groupby('banner')['roas'].median().sort_values(ascending=False).head(5)
    ax = setup_plot('Top 5 Banners by Median Daily ROAS', ylabel='Median ROAS')
    median_roas_banner.plot(kind='bar', ax=ax)
    plt.xticks(rotation=45, ha='right')
    finalize_plot('Top 5 Banners by Median Daily ROAS', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q15: Conversions by placement?
    placement_conversions = df_kpi.groupby('placement')['conversions'].sum().sort_values(ascending=False).head(10)
    ax = setup_plot('Top 10 Placements by Total Conversions', ylabel='Total Conversions')
    placement_conversions.plot(kind='bar', ax=ax)
    plt.xticks(rotation=45, ha='right')
    finalize_plot('Top 10 Placements by Total Conversions', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q16: Weekday vs weekend performance?
    df_kpi['day_type'] = np.where(df_kpi['date'].dt.dayofweek < 5, 'Weekday', 'Weekend')
    comparison = df_kpi.groupby('day_type')[['ctr', 'cpc', 'cvr']].mean()
    q16_text = "Average Performance: Weekday vs. Weekend\n" + comparison.to_markdown(floatfmt=".4f")
    logging.info(q16_text)
    add_text_to_pdf(q16_text, pdf_report)

    #Q17: CPC by campaign and banner?
    cpc_pivot = df_kpi.groupby(['campaign_number', 'banner'])['cpc'].mean().unstack()
    cpc_pivot.plot(kind='bar', subplots=True, layout=(3,3), figsize=(12, 10), legend=False, sharey=True)
    plt.suptitle('Average CPC by Banner for Each Campaign', y=1.02, fontsize=16, fontweight='bold')
    finalize_plot('Average CPC by Banner for Each Campaign', save_plots=SAVE_PLOTS, pdf_pages=pdf_report)

    #Q18: Cost-effective conversions (CPA)?
    cpa_df = df_kpi.groupby(['banner', 'placement']).agg(
        total_cost=('cost', 'sum'),
        total_conversions=('conversions', 'sum')
    ).reset_index()
    cpa_df['cpa'] = cpa_df['total_cost'] / cpa_df['total_conversions'].replace(0, np.nan)
    cost_effective = cpa_df.dropna().sort_values('cpa').head(5)
    q18_text = "Top 5 Most Cost-Effective Combos by CPA\n" + cost_effective[['banner', 'placement', 'cpa']].to_markdown(index=False, floatfmt=".2f")
    logging.info(q18_text)
    add_text_to_pdf(q18_text, pdf_report)

    #Q20: Effectiveness by engagement type?
    engagement_effectiveness = df_kpi.groupby('user_engagement')[['cvr', 'roas']].mean().sort_values('roas', ascending=False)
    q20_text = "Effectiveness by Engagement Type\n" + engagement_effectiveness.to_markdown(floatfmt=".2%")
    logging.info(q20_text)
    add_text_to_pdf(q20_text, pdf_report)

    analysis_time = time.time() - analysis_start_time
    logging.info(f"Business Analysis (20 Questions) took {analysis_time:.2f} seconds.")

**Synergy Score & Bandit Simulation**

In [41]:
# A custom score to find optimal banner/placement pairs and a simulation to estimate the impact of budget reallocation.
if not df.empty:
    twist_start_time = time.time()

    #Synergy Score: z(RPC) + z(CVR) − z(CPC)
    combo_perf = df_kpi.groupby(['banner', 'placement']).agg(
        avg_rpc=('rpc', 'mean'), avg_cvr=('cvr', 'mean'), avg_cpc=('cpc', 'mean'),
        total_spend=('cost', 'sum'), total_sales=('sales', 'sum')
    ).reset_index()

    for kpi in ['avg_rpc', 'avg_cvr', 'avg_cpc']:
        mean = combo_perf[kpi].mean()
        std = combo_perf[kpi].std()
        combo_perf[f'z_{kpi}'] = (combo_perf[kpi] - mean) / std if std > 0 else 0

    combo_perf['synergy_score'] = (combo_perf['z_avg_rpc'] + combo_perf['z_avg_cvr'] - combo_perf['z_avg_cpc']).fillna(0)

    top_synergy = combo_perf.sort_values('synergy_score', ascending=False).head()[['banner', 'placement', 'synergy_score']]
    synergy_text = "Top 5 Banner/Placement Combos by Synergy Score\n" + top_synergy.to_markdown(index=False)
    logging.info(synergy_text)
    add_text_to_pdf(synergy_text, pdf_report)

    #Beginner Bandit Simulation
    spend_to_reallocate = combo_perf['total_spend'].sum() * 0.10
    bottom_20_percentile = combo_perf['synergy_score'].quantile(0.2)
    top_20_percentile = combo_perf['synergy_score'].quantile(0.8)

    top_performers = combo_perf[combo_perf['synergy_score'] >= top_20_percentile]
    avg_roas_top = (top_performers['total_sales'].sum() / top_performers['total_spend'].sum()) if top_performers['total_spend'].sum() > 0 else 0

    estimated_sales_gain = spend_to_reallocate * avg_roas_top
    original_total_sales = df_kpi['sales'].sum()
    estimated_uplift_percent = ((estimated_sales_gain - spend_to_reallocate) / original_total_sales) * 100 if original_total_sales > 0 else 0

    simulation_text = (
        f"Budget Reallocation Simulation\n"
        f"Reallocating ${spend_to_reallocate:,.2f} (10% of spend)...\n"
        f"Estimated Sales Gain: ${estimated_sales_gain - spend_to_reallocate:,.2f}\n"
        f"Estimated Uplift: {estimated_uplift_percent:.2f}%"
    )
    logging.info(simulation_text)
    add_text_to_pdf(simulation_text, pdf_report)

    #Export Top Actions
    bottom_performers = combo_perf[combo_perf['synergy_score'] <= bottom_20_percentile]
    pause_candidates = bottom_performers[['banner', 'placement', 'synergy_score']].sort_values('synergy_score').head()
    scale_candidates = top_performers[['banner', 'placement', 'synergy_score']].sort_values('synergy_score', ascending=False).head()

    actions_df = pd.DataFrame({
        'Action': ['Scale'] * 5 + ['Pause'] * 5,
        'Banner': list(scale_candidates['banner']) + list(pause_candidates['banner']),
        'Placement': list(scale_candidates['placement']) + list(pause_candidates['placement']),
        'Synergy Score': list(scale_candidates['synergy_score']) + list(pause_candidates['synergy_score']),
        'Estimated Impact': f"Part of a 10% budget shift strategy estimated to generate a {estimated_uplift_percent:.2f}% sales uplift."
    })

    actions_df.to_csv(OUTPUT_FILENAME, index=False)
    logging.info(f"'{OUTPUT_FILENAME}' has been exported with scaling and pausing recommendations.")

    twist_time = time.time() - twist_start_time
    logging.info(f"Synergy Score & Simulation took {twist_time:.2f} seconds.")

**Finalize and Save Report**

In [42]:
if not df.empty:
    # Add the final summary and recommendations page
    summary_text = """
    Executive Summary:
    1. This report details the performance of online advertising campaigns from April to June 2020.
    2. The analysis reveals several key drivers of success, most notably the immense value of the
    'High' user engagement segment, which accounts for nearly 80% of all revenue. Performance
    is strongest mid-week, and specific combinations of ad placements and banner sizes, such as
    the 300x250 banner on placement 'ghi', deliver exceptionally high returns. A "Synergy Score"
    was developed to identify high-potential ad combinations, and a simulation suggests that
    reallocating just 10% of the budget from the worst to the best performers could yield a
    significant sales uplift.
    ---------------------------------------------------------------------------------------------
    Strategic Recommendations & Next Steps:
    1. Target High-Engagement Users: This segment provides ~80% of revenue and converts
        effectively across all formats. Focus budget and creative efforts here.
    2. Optimize Placements: 'ghi' and 'mno' are top for volume and conversions. Prioritize
        these placements and investigate what makes them successful to find similar opportunities.
        The 300x250 banner on 'ghi' is a star performer with a high ROAS.
    3. Leverage Top Creatives: The 240x400 and 728x90 banners drive the most clicks, while
        the 580x400 banner shows the most consistent day-to-day ROI. Ensure these are always
        active in campaigns.
    4. Mid-Week Conversion Push: Conversion rates peak from Tuesday to Thursday. Consider
        increasing bids or launching promotions during this window to maximize results.
   ----------------------------------------------------------------------------------------------
    Potential Next Steps:
    1. A/B Testing: Formally test the banner/placement combinations identified by the Synergy Score
        to validate their performance.
    2. Predictive Modeling: Build a model to predict the ROAS of a campaign based on features
        like placement, banner size, and target audience.
    3. Interactive Dashboard: Develop a dashboard (using tools like Streamlit or Panel) to allow
        marketing teams to explore this data and filter by campaign, date, or other dimensions
        interactively.
    """
    add_text_to_pdf(summary_text, pdf_report)
    # Close and save the PDF file
    pdf_report.close()
    logging.info(f"Successfully generated and saved the full report to: {REPORT_FILENAME}")
    # Print a final summary to the console for a quick overview
    print("\n" + "="*80)
    print(" " * 25 + "ADVERTISING ANALYSIS COMPLETE")
    print("="*80)
    print(f"A comprehensive PDF report has been saved as: {REPORT_FILENAME}")
    print(f"A CSV with actionable recommendations has been saved as: {OUTPUT_FILENAME}")
    print("\nKEY STRATEGIC RECOMMENDATIONS")
    print("1. Target High-Engagement Users: This segment provides ~80% of revenue.")
    print("2. Optimize Placements: 'ghi' and 'mno' are top for volume and conversions.")
    print("3. Leverage Top Creatives: 240x400 and 728x90 banners drive the most clicks.")
    print("4. Mid-Week Conversion Push: CVRs peak from Tuesday to Thursday.")
    print("="*80)
logging.info("Notebook execution finished.")


                         ADVERTISING ANALYSIS COMPLETE
A comprehensive PDF report has been saved as: Advertising_Performance_Report.pdf
A CSV with actionable recommendations has been saved as: top_actions.csv

KEY STRATEGIC RECOMMENDATIONS
1. Target High-Engagement Users: This segment provides ~80% of revenue.
2. Optimize Placements: 'ghi' and 'mno' are top for volume and conversions.
3. Leverage Top Creatives: 240x400 and 728x90 banners drive the most clicks.
4. Mid-Week Conversion Push: CVRs peak from Tuesday to Thursday.
